In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neural_network import MLPRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mbb2024advancedstats/Barttorvik - Sheet1.csv
/kaggle/input/marchmadnessresults/Shooting Splits.csv
/kaggle/input/marchmadnessresults/538 Ratings.csv
/kaggle/input/marchmadnessresults/Barttorvik Away-Neutral.csv
/kaggle/input/marchmadnessresults/Heat Check Tournament Index.csv
/kaggle/input/marchmadnessresults/Conference Stats Neutral.csv
/kaggle/input/marchmadnessresults/Barttorvik Away.csv
/kaggle/input/marchmadnessresults/Resumes.csv
/kaggle/input/marchmadnessresults/Preseason Votes.csv
/kaggle/input/marchmadnessresults/Barttorvik Neutral.csv
/kaggle/input/marchmadnessresults/Conference Results.csv
/kaggle/input/marchmadnessresults/Conference Stats.csv
/kaggle/input/marchmadnessresults/Upset Count.csv
/kaggle/input/marchmadnessresults/Conference Stats Home.csv
/kaggle/input/marchmadnessresults/Conference Stats Away.csv
/kaggle/input/marchmadnessresults/Barttorvik Home.csv
/kaggle/input/marchmadnessresults/Public Picks.csv
/kaggle/input/marchmadnessresults/Tournament Loc

In [2]:
from sklearn import preprocessing

In [3]:
AllData = pd.read_parquet(f"/kaggle/input/barttorvikdata/BartTovikData/2008.parquet")
for i in range(2008, 2024):
    if i == 2020:
        continue
    temp = pd.read_parquet(f"/kaggle/input/barttorvikdata/BartTovikData/{i}.parquet")
    AllData = pd.concat([AllData, temp])

In [4]:
FinalData = AllData.drop(['Record', 'Barthag', 'Games', 'Avg Hgt.', 'Raw T', 'PAKE', 'Talent'], axis = 1)

In [5]:
FinalData = FinalData.loc[FinalData['PASE'] != 0.0]

In [6]:
FinalData = FinalData.groupby(['Team', 'Year']).sum().reset_index()

In [7]:
temp = FinalData.drop(['Team', 'Year'], axis = 1)

In [8]:
scaler = preprocessing.StandardScaler().fit(temp)
ScaledData = scaler.transform(temp)
colsToSwap = (1, 4, 6, 8, 12, 14, 16, 18, 20, 25, 27)
test = ScaledData
test[:, colsToSwap] *= -1

In [9]:
for i, col in enumerate(temp.columns.values):
    print(f"{i} {col}")

0 Adj OE
1 Adj DE
2 Wins
3 eFG
4 eFG D.
5 FT Rate
6 FT Rate D
7 TOV%
8 TOV% D
9 O Reb%
10 Op OReb%
11 2P %
12 2P % D.
13 3P %
14 3P % D.
15 Blk %
16 Blked %
17 Ast %
18 Op Ast %
19 3P Rate
20 3P Rate D
21 Adj. T
22 Eff. Hgt.
23 Exp.
24 PASE
25 FT%
26 Op. FT%
27 PPP Off.
28 PPP Def.
29 Elite SOS


In [10]:
weights = np.array([10, 10, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 70/26, 10, 70/26, 70/26, 70/26, 70/26, 70/26])
teamTotals = np.sum(test * weights, axis = 1)
# teamTotals = np.sum(test, axis = 1)

In [11]:
FinalData['Score'] = teamTotals.tolist()

In [12]:
FinalData

,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Adj. T,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score
0,Abilene Christian,2019,102.2,101.0,23,52.4,49.6,34.5,38.4,17.4,...,66.3,79.024,2.044,-0.105,72.3,72.3,1.068,0.967,7.995,-20.118060
1,Abilene Christian,2021,100.9,92.6,19,52.9,45.4,34.0,42.5,19.4,...,70.8,79.765,2.228,0.842,68.8,69.1,1.066,0.862,9.499,-5.857027
2,Akron,2009,101.1,94.0,22,48.2,45.2,36.4,45.0,20.9,...,66.0,79.718,1.461,-0.250,71.5,69.2,1.010,0.920,14.565,-15.594872
3,Akron,2011,101.4,98.6,22,49.9,46.5,31.9,38.2,17.7,...,67.1,80.897,1.985,-0.105,70.8,66.3,1.029,0.975,13.032,-11.789382
4,Akron,2013,106.2,94.9,23,51.4,44.5,35.7,35.9,20.2,...,66.7,81.274,1.730,-0.507,65.2,71.5,1.074,0.932,14.351,-3.782162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,Xavier,2018,121.4,97.3,28,55.7,50.0,40.2,24.8,17.0,...,71.6,80.776,1.844,-2.289,79.0,75.0,1.155,1.021,31.853,-19.256722
956,Xavier,2023,118.6,98.4,25,55.6,50.4,30.8,27.1,17.5,...,71.2,81.528,2.432,0.151,71.4,72.4,1.129,1.027,31.866,3.187882
957,Yale,2016,108.2,95.3,21,52.5,46.3,42.1,33.8,20.1,...,66.8,79.410,2.270,0.493,66.7,69.4,1.107,0.944,15.940,3.142615
958,Yale,2019,110.1,102.0,21,56.1,47.2,30.5,30.6,18.1,...,71.7,80.354,2.113,-0.158,73.3,73.7,1.099,1.002,14.891,-7.191490


In [13]:
X = FinalData.drop("Score", axis = 1)

In [14]:
X_test = FinalData[FinalData["Year"] >= 2021]
X_test

,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Adj. T,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score
1,Abilene Christian,2021,100.9,92.6,19,52.9,45.4,34.0,42.5,19.4,...,70.8,79.765,2.228,0.842,68.8,69.1,1.066,0.862,9.499,-5.857027
5,Akron,2022,104.8,102.7,22,52.8,49.3,40.1,26.2,17.7,...,63.0,80.080,1.974,-0.250,69.2,70.3,1.082,1.002,9.638,-11.252248
8,Alabama,2021,110.7,88.6,24,51.3,45.3,28.6,31.9,18.6,...,74.8,80.152,1.841,-0.329,72.0,68.6,1.059,0.929,31.996,-3.183144
9,Alabama,2022,115.9,98.1,19,51.5,49.4,33.6,35.2,19.8,...,72.8,79.948,1.646,-1.059,73.0,71.2,1.087,1.039,34.814,-9.055832
10,Alabama,2023,115.4,88.3,29,52.7,41.5,36.7,32.8,19.0,...,73.5,82.068,1.130,-1.289,72.6,71.4,1.106,0.922,33.700,1.515220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,Wisconsin,2021,111.5,90.3,17,49.4,47.9,26.9,28.5,13.5,...,65.9,81.638,2.316,0.388,76.7,71.2,1.050,0.970,38.060,-4.245329
938,Wisconsin,2022,110.1,93.8,24,48.2,49.4,32.3,27.7,12.7,...,67.5,82.342,1.462,-0.849,74.2,68.0,1.053,0.991,31.781,-20.935575
945,Wright St.,2022,104.2,106.9,20,52.0,51.2,31.1,26.0,17.9,...,69.6,79.768,1.581,-0.013,76.6,71.3,1.086,1.035,8.922,-18.018570
956,Xavier,2023,118.6,98.4,25,55.6,50.4,30.8,27.1,17.5,...,71.2,81.528,2.432,0.151,71.4,72.4,1.129,1.027,31.866,3.187882


In [15]:
y_test = X_test.Score.to_numpy()
y_test 

array([-5.85702677e+00, -1.12522479e+01, -3.18314433e+00, -9.05583156e+00,
        1.51521952e+00,  1.45898110e+00, -1.19477350e+01, -9.39505534e+00,
        6.21563639e+00,  8.52125359e+00,  1.02214107e+01, -7.42323363e+00,
        5.27921309e+00, -8.05397273e+00,  2.46559588e+01, -1.64052637e+01,
       -9.82233735e+00, -7.73780318e+00, -1.47162256e+01, -2.53401628e+01,
       -1.46143258e+01, -2.06866401e+01, -1.95231277e+01, -1.06113159e+01,
       -1.07032384e+01, -1.28652605e+01, -1.08078848e+01, -3.73438657e+00,
       -2.07300894e+01, -9.25657486e+00, -7.53432180e+00,  4.93770707e+01,
        8.69129746e+00,  1.67788116e+00,  1.99306389e+01, -1.50014602e+01,
       -1.03390978e+01, -1.81897621e+01, -1.53451720e+01, -1.81229348e+01,
        1.97057989e+01, -2.98890338e+00, -1.40513527e+01, -2.85204431e+01,
       -4.46371722e+00,  3.24431967e+01,  7.93367503e+00, -1.43459569e+00,
       -8.73922486e+00, -2.22955483e+01, -1.32398298e+01,  2.05514596e+01,
        1.54539618e+00,  

In [16]:
X_train = FinalData[FinalData["Year"] < 2021]
X_train

,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Adj. T,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score
0,Abilene Christian,2019,102.2,101.0,23,52.4,49.6,34.5,38.4,17.4,...,66.3,79.024,2.044,-0.105,72.3,72.3,1.068,0.967,7.995,-20.118060
2,Akron,2009,101.1,94.0,22,48.2,45.2,36.4,45.0,20.9,...,66.0,79.718,1.461,-0.250,71.5,69.2,1.010,0.920,14.565,-15.594872
3,Akron,2011,101.4,98.6,22,49.9,46.5,31.9,38.2,17.7,...,67.1,80.897,1.985,-0.105,70.8,66.3,1.029,0.975,13.032,-11.789382
4,Akron,2013,106.2,94.9,23,51.4,44.5,35.7,35.9,20.2,...,66.7,81.274,1.730,-0.507,65.2,71.5,1.074,0.932,14.351,-3.782162
6,Alabama,2012,105.0,88.1,21,49.0,43.4,36.5,38.6,20.4,...,63.1,79.579,1.010,-0.612,71.5,65.1,1.022,0.915,26.959,-15.168671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,Xavier,2016,117.1,94.5,27,51.5,47.9,42.9,34.4,17.5,...,72.6,80.569,1.534,-1.329,73.1,70.4,1.117,0.976,29.508,-13.614262
954,Xavier,2017,114.5,97.3,21,51.6,51.8,41.2,33.7,18.6,...,68.7,80.220,1.708,2.362,68.8,69.9,1.087,1.040,32.410,12.980347
955,Xavier,2018,121.4,97.3,28,55.7,50.0,40.2,24.8,17.0,...,71.6,80.776,1.844,-2.289,79.0,75.0,1.155,1.021,31.853,-19.256722
957,Yale,2016,108.2,95.3,21,52.5,46.3,42.1,33.8,20.1,...,66.8,79.410,2.270,0.493,66.7,69.4,1.107,0.944,15.940,3.142615


In [17]:
y_train = X_train.Score.to_numpy()
y_train

array([-2.01180605e+01, -1.55948723e+01, -1.17893823e+01, -3.78216154e+00,
       -1.51686709e+01, -3.93920650e+00, -1.63894472e+01, -1.77466994e+01,
       -2.67963937e+01,  2.98807275e+01, -8.79525536e+00, -8.63014805e+00,
        5.65567640e+01,  9.18741837e+00,  1.46102396e+01,  6.63500315e+00,
        4.28085660e+00,  9.39150982e+00, -3.75985379e+00, -4.73241069e+00,
       -1.38925604e+01,  4.38153134e+00, -7.05651230e+00, -1.89405732e+01,
        7.25349253e+01, -3.24293630e+00,  7.77408605e-01, -1.29425549e+01,
       -2.29620125e+01, -9.34843540e+00,  2.38532929e+01,  2.03668117e+01,
       -2.04467390e+01,  6.85980311e+01, -7.34972439e+00, -1.66251544e+00,
        2.13871045e+00, -9.53524262e+00, -1.58857419e+01,  4.03176663e+01,
        2.12520242e+01,  1.31620492e+01,  1.48761280e+01, -1.68073485e+01,
       -1.64793873e+01,  1.06100278e+01, -4.08378992e+00,  2.94657100e+01,
       -1.10438758e+01, -8.25313714e+00, -1.46419517e+01, -1.71833022e+01,
       -1.37531496e+01, -

In [18]:
final_train = X_train.drop(["Score", "Team", "Year"], axis = 1).to_numpy()
final_train

array([[102.2  , 101.   ,  23.   , ...,   1.068,   0.967,   7.995],
       [101.1  ,  94.   ,  22.   , ...,   1.01 ,   0.92 ,  14.565],
       [101.4  ,  98.6  ,  22.   , ...,   1.029,   0.975,  13.032],
       ...,
       [121.4  ,  97.3  ,  28.   , ...,   1.155,   1.021,  31.853],
       [108.2  ,  95.3  ,  21.   , ...,   1.107,   0.944,  15.94 ],
       [110.1  , 102.   ,  21.   , ...,   1.099,   1.002,  14.891]])

In [19]:
final_test = X_test.drop(["Score", "Team", "Year"], axis = 1).to_numpy()
final_test 

array([[100.9  ,  92.6  ,  19.   , ...,   1.066,   0.862,   9.499],
       [104.8  , 102.7  ,  22.   , ...,   1.082,   1.002,   9.638],
       [110.7  ,  88.6  ,  24.   , ...,   1.059,   0.929,  31.996],
       ...,
       [104.2  , 106.9  ,  20.   , ...,   1.086,   1.035,   8.922],
       [118.6  ,  98.4  ,  25.   , ...,   1.129,   1.027,  31.866],
       [ 99.8  ,  98.4  ,  18.   , ...,   1.017,   0.988,  14.644]])

In [20]:
regr = MLPRegressor(random_state = 1, activation = 'logistic', max_iter = 500).fit(final_train, y_train)

In [21]:
output = regr.predict(final_test)

In [22]:
y_test

array([-5.85702677e+00, -1.12522479e+01, -3.18314433e+00, -9.05583156e+00,
        1.51521952e+00,  1.45898110e+00, -1.19477350e+01, -9.39505534e+00,
        6.21563639e+00,  8.52125359e+00,  1.02214107e+01, -7.42323363e+00,
        5.27921309e+00, -8.05397273e+00,  2.46559588e+01, -1.64052637e+01,
       -9.82233735e+00, -7.73780318e+00, -1.47162256e+01, -2.53401628e+01,
       -1.46143258e+01, -2.06866401e+01, -1.95231277e+01, -1.06113159e+01,
       -1.07032384e+01, -1.28652605e+01, -1.08078848e+01, -3.73438657e+00,
       -2.07300894e+01, -9.25657486e+00, -7.53432180e+00,  4.93770707e+01,
        8.69129746e+00,  1.67788116e+00,  1.99306389e+01, -1.50014602e+01,
       -1.03390978e+01, -1.81897621e+01, -1.53451720e+01, -1.81229348e+01,
        1.97057989e+01, -2.98890338e+00, -1.40513527e+01, -2.85204431e+01,
       -4.46371722e+00,  3.24431967e+01,  7.93367503e+00, -1.43459569e+00,
       -8.73922486e+00, -2.22955483e+01, -1.32398298e+01,  2.05514596e+01,
        1.54539618e+00,  

In [23]:
results = X_test
results['Results'] = output
results.sort_values('Results', ascending=False).head(50)

/tmp/ipykernel_19/204296415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['Results'] = output


,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score,Results
148,Connecticut,2023,118.9,92.5,25,53.5,45.5,31.4,38.7,18.9,...,81.025,1.724,4.454,75.4,73.6,1.144,0.949,29.244,49.377071,34.442345
61,Baylor,2021,123.3,93.5,22,57.5,49.1,27.0,31.7,17.6,...,79.370,1.967,2.711,69.7,68.6,1.196,0.941,27.785,24.655959,32.946682
330,Kansas,2022,120.2,92.9,28,54.1,46.9,32.8,27.8,17.8,...,80.111,2.082,2.711,72.4,70.7,1.119,0.970,36.390,34.131410,30.779951
209,Florida Atlantic,2023,112.2,94.8,29,55.0,45.9,29.5,26.7,16.7,...,79.224,2.036,3.388,71.8,74.9,1.129,0.943,16.095,32.443197,30.041122
690,San Diego St.,2023,111.3,90.1,26,50.1,47.5,34.3,29.4,17.6,...,80.193,2.718,3.842,73.6,73.6,1.064,0.952,25.602,35.893477,28.696594
527,North Carolina,2022,112.8,95.9,24,52.1,49.8,29.6,21.5,16.5,...,81.474,1.742,4.276,77.2,73.7,1.096,1.014,29.659,34.804972,27.308653
825,USC,2021,113.3,90.2,22,52.1,45.4,37.5,24.8,18.1,...,82.589,1.679,1.941,64.7,72.8,1.089,0.948,32.707,24.481017,26.729321
275,Houston,2022,117.1,88.9,29,53.6,43.5,28.7,35.1,17.1,...,80.470,2.179,1.842,66.9,70.5,1.147,0.890,22.668,25.443152,23.699085
803,UCLA,2021,114.2,96.2,17,52.0,50.7,32.6,28.0,16.7,...,79.632,1.376,3.362,72.1,73.6,1.083,1.019,34.411,19.569168,22.615825
190,Duke,2022,120.4,94.4,28,55.6,47.0,28.6,18.9,14.9,...,81.821,0.988,1.671,72.9,67.2,1.169,0.979,28.362,19.705799,21.987588


In [24]:
results

,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score,Results
1,Abilene Christian,2021,100.9,92.6,19,52.9,45.4,34.0,42.5,19.4,...,79.765,2.228,0.842,68.8,69.1,1.066,0.862,9.499,-5.857027,2.913727
5,Akron,2022,104.8,102.7,22,52.8,49.3,40.1,26.2,17.7,...,80.080,1.974,-0.250,69.2,70.3,1.082,1.002,9.638,-11.252248,-8.643952
8,Alabama,2021,110.7,88.6,24,51.3,45.3,28.6,31.9,18.6,...,80.152,1.841,-0.329,72.0,68.6,1.059,0.929,31.996,-3.183144,1.224311
9,Alabama,2022,115.9,98.1,19,51.5,49.4,33.6,35.2,19.8,...,79.948,1.646,-1.059,73.0,71.2,1.087,1.039,34.814,-9.055832,-11.555451
10,Alabama,2023,115.4,88.3,29,52.7,41.5,36.7,32.8,19.0,...,82.068,1.130,-1.289,72.6,71.4,1.106,0.922,33.700,1.515220,-1.625512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,Wisconsin,2021,111.5,90.3,17,49.4,47.9,26.9,28.5,13.5,...,81.638,2.316,0.388,76.7,71.2,1.050,0.970,38.060,-4.245329,-10.916023
938,Wisconsin,2022,110.1,93.8,24,48.2,49.4,32.3,27.7,12.7,...,82.342,1.462,-0.849,74.2,68.0,1.053,0.991,31.781,-20.935575,-23.156993
945,Wright St.,2022,104.2,106.9,20,52.0,51.2,31.1,26.0,17.9,...,79.768,1.581,-0.013,76.6,71.3,1.086,1.035,8.922,-18.018570,-15.971392
956,Xavier,2023,118.6,98.4,25,55.6,50.4,30.8,27.1,17.5,...,81.528,2.432,0.151,71.4,72.4,1.129,1.027,31.866,3.187882,-0.937011


In [25]:
output2 = regr.predict(final_train)
output2

array([-16.43792472,  -4.44406204,  -7.83337237,  -1.70129998,
       -13.68254197,  -3.07985017,  -6.86198726, -15.23469136,
       -19.31832506,   0.67802746,  -0.40945734,  -0.42659772,
        25.69021471,  23.05150767,  22.07430567,  14.79035163,
         3.49729806,   7.12974954, -10.24022847,  -6.78932898,
       -20.07338275,  12.91268416,  -9.12137661, -19.24420056,
        29.16127344,  -5.92383931,  -8.27846908, -16.75436839,
       -20.13651359, -13.83646872,  28.72717207,   2.51681805,
       -13.34863071,  26.20459824, -11.21491407,   2.5584772 ,
        -1.97532533, -10.66601575, -18.85085751,  -5.03536648,
        26.82803144,  24.35684515,  20.05870514,  -7.46767035,
       -15.22015662,  12.66722012,  -1.31622275,   7.76769147,
        -0.82081178,  -1.3635783 ,  -6.15346914,  -5.25956787,
       -16.97684233, -14.85802227,  17.91304953,  -9.79700723,
       -10.47672274, -17.10910838, -13.9417569 , -16.84085726,
        -6.03211842, -15.33415507, -16.65171214, -18.46

In [26]:
results2 = X_train
results2
results2["Results"] = output2
results2

/tmp/ipykernel_19/2565301647.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2["Results"] = output2


,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score,Results
0,Abilene Christian,2019,102.2,101.0,23,52.4,49.6,34.5,38.4,17.4,...,79.024,2.044,-0.105,72.3,72.3,1.068,0.967,7.995,-20.118060,-16.437925
2,Akron,2009,101.1,94.0,22,48.2,45.2,36.4,45.0,20.9,...,79.718,1.461,-0.250,71.5,69.2,1.010,0.920,14.565,-15.594872,-4.444062
3,Akron,2011,101.4,98.6,22,49.9,46.5,31.9,38.2,17.7,...,80.897,1.985,-0.105,70.8,66.3,1.029,0.975,13.032,-11.789382,-7.833372
4,Akron,2013,106.2,94.9,23,51.4,44.5,35.7,35.9,20.2,...,81.274,1.730,-0.507,65.2,71.5,1.074,0.932,14.351,-3.782162,-1.701300
6,Alabama,2012,105.0,88.1,21,49.0,43.4,36.5,38.6,20.4,...,79.579,1.010,-0.612,71.5,65.1,1.022,0.915,26.959,-15.168671,-13.682542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,Xavier,2016,117.1,94.5,27,51.5,47.9,42.9,34.4,17.5,...,80.569,1.534,-1.329,73.1,70.4,1.117,0.976,29.508,-13.614262,-16.161158
954,Xavier,2017,114.5,97.3,21,51.6,51.8,41.2,33.7,18.6,...,80.220,1.708,2.362,68.8,69.9,1.087,1.040,32.410,12.980347,13.376356
955,Xavier,2018,121.4,97.3,28,55.7,50.0,40.2,24.8,17.0,...,80.776,1.844,-2.289,79.0,75.0,1.155,1.021,31.853,-19.256722,-22.639893
957,Yale,2016,108.2,95.3,21,52.5,46.3,42.1,33.8,20.1,...,79.410,2.270,0.493,66.7,69.4,1.107,0.944,15.940,3.142615,4.789042


In [27]:
final_predicted = pd.concat([results, results2])

In [28]:
final_predicted

,Team,Year,Adj OE,Adj DE,Wins,eFG,eFG D.,FT Rate,FT Rate D,TOV%,...,Eff. Hgt.,Exp.,PASE,FT%,Op. FT%,PPP Off.,PPP Def.,Elite SOS,Score,Results
1,Abilene Christian,2021,100.9,92.6,19,52.9,45.4,34.0,42.5,19.4,...,79.765,2.228,0.842,68.8,69.1,1.066,0.862,9.499,-5.857027,2.913727
5,Akron,2022,104.8,102.7,22,52.8,49.3,40.1,26.2,17.7,...,80.080,1.974,-0.250,69.2,70.3,1.082,1.002,9.638,-11.252248,-8.643952
8,Alabama,2021,110.7,88.6,24,51.3,45.3,28.6,31.9,18.6,...,80.152,1.841,-0.329,72.0,68.6,1.059,0.929,31.996,-3.183144,1.224311
9,Alabama,2022,115.9,98.1,19,51.5,49.4,33.6,35.2,19.8,...,79.948,1.646,-1.059,73.0,71.2,1.087,1.039,34.814,-9.055832,-11.555451
10,Alabama,2023,115.4,88.3,29,52.7,41.5,36.7,32.8,19.0,...,82.068,1.130,-1.289,72.6,71.4,1.106,0.922,33.700,1.515220,-1.625512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,Xavier,2016,117.1,94.5,27,51.5,47.9,42.9,34.4,17.5,...,80.569,1.534,-1.329,73.1,70.4,1.117,0.976,29.508,-13.614262,-16.161158
954,Xavier,2017,114.5,97.3,21,51.6,51.8,41.2,33.7,18.6,...,80.220,1.708,2.362,68.8,69.9,1.087,1.040,32.410,12.980347,13.376356
955,Xavier,2018,121.4,97.3,28,55.7,50.0,40.2,24.8,17.0,...,80.776,1.844,-2.289,79.0,75.0,1.155,1.021,31.853,-19.256722,-22.639893
957,Yale,2016,108.2,95.3,21,52.5,46.3,42.1,33.8,20.1,...,79.410,2.270,0.493,66.7,69.4,1.107,0.944,15.940,3.142615,4.789042


In [29]:
scores = final_predicted.groupby("Team").mean().reset_index()

In [30]:
scores = scores[["Team", "Results"]]
scores

,Team,Results
0,Abilene Christian,-6.762099
1,Akron,-5.655672
2,Alabama,-5.743809
3,Albany,-13.805001
4,American,-0.052676
...,...,...
231,Wofford,-10.731004
232,Wright St.,-16.626578
233,Wyoming,-10.102385
234,Xavier,1.942151


In [31]:
scores.describe()

,Results
count,236.000000
mean,-4.843113
std,9.875194
min,-22.553047
25%,-12.307116
50%,-5.820082
75%,1.967567
max,32.833930


In [32]:
matchups = pd.read_csv('/kaggle/input/marchmadnessresults/Tournament Matchups.csv')

In [33]:
matchups = matchups[matchups['YEAR'] == 2024]

In [34]:
matchups

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE
0,2024,2036,0,1067,Connecticut,1,1,64,NaN
1,2024,2035,0,1026,Stetson,16,64,64,NaN
2,2024,2034,0,1038,North Carolina,1,1,64,NaN
3,2024,2033,0,1055,Howard,16,64,64,NaN
4,2024,2032,0,1038,North Carolina,1,1,64,NaN
...,...,...,...,...,...,...,...,...,...
143,2024,1893,0,1013,Wisconsin,5,16,16,NaN
144,2024,1892,0,1033,Purdue,1,1,16,NaN
145,2024,1891,0,1051,Kansas,4,16,16,NaN
146,2024,1890,0,1033,Purdue,1,1,16,NaN


In [35]:
submission = pd.DataFrame(columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])

In [36]:
# EAST
east = [[1,'M',1,'R1W1','W01'],
[2,'M',1,'R1W8','W08'],
[3,'M',1,'R1W5','W05'],
[4,'M',1,'R1W4','W04'],
[5,'M',1,'R1W6','W06'],
[6,'M',1,'R1W3','W03'],
[7,'M',1,'R1W7','W07'],
[8,'M',1,'R1W2','W02'],

[9,'M',1,'R2W1','W08'],
[10,'M',1,'R2W4','W04'],
[11,'M',1,'R2W3','W03'],
[12,'M',1,'R2W2','W07'],

[13,'M',1,'R3W1','W08'],
[14,'M',1,'R3W2','W07'],

[15,'M',1,'R4W1','W07']]

# WEST
west = [[16,'M',1,'R1X1','X01'],
[17,'M',1,'R1X8','X08'],
[18,'M',1,'R1X5','X05'],
[19,'M',1,'R1X4','X04'],
[20,'M',1,'R1X6','X06'],
[21,'M',1,'R1X3','X03'],
[22,'M',1,'R1X7','X07'],
[23,'M',1,'R1X2','X02'],

[24,'M',1,'R2X1','X08'],
[25,'M',1,'R2X4','X05'],
[26,'M',1,'R2X3','X06'],
[27,'M',1,'R2X2','X02'],

[28,'M',1,'R3X1','X08'],
[29,'M',1,'R3X2','X06'],

[30,'M',1,'R4X1','X08']]

# SOUTH
south = [[31,'M',1,'R1Y1','Y01'],
[32,'M',1,'R1Y8','Y09'],
[33,'M',1,'R1Y5','Y05'],
[34,'M',1,'R1Y4','Y04'],
[35,'M',1,'R1Y6','Y06'],
[36,'M',1,'R1Y3','Y03'],
[37,'M',1,'R1Y7','Y07'],
[38,'M',1,'R1Y2','Y15'],

[39,'M',1,'R2Y1','Y09'],
[40,'M',1,'R2Y4','Y04'],
[41,'M',1,'R2Y3','Y03'],
[42,'M',1,'R2Y2','Y07'],

[43,'M',1,'R3Y1','Y04'],
[44,'M',1,'R3Y2','Y03'],

[45,'M',1,'R4Y1','Y03']]

# MIDWEST
midwest = [[46,'M',1,'R1Z1','Z01'],
[47,'M',1,'R1Z8','Z09'],
[48,'M',1,'R1Z5','Z05'],
[49,'M',1,'R1Z4','Z04'],
[50,'M',1,'R1Z6','Z06'],
[51,'M',1,'R1Z3','Z03'],
[52,'M',1,'R1Z7','Z07'],
[53,'M',1,'R1Z2','Z15'],

[54,'M',1,'R2Z1','Z01'],
[55,'M',1,'R2Z4','Z05'],
[56,'M',1,'R2Z3','Z06'],
[57,'M',1,'R2Z2','Z15'],

[58,'M',1,'R3Z1','Z05'],
[59,'M',1,'R3Z2','Z06'],

[60,'M',1,'R4Z1','Z06']]

# FINAL FOUR
ff = [[61,'M',1,'R5WX','W07'],
[62,'M',1,'R5YZ','Z06'],
[63,'M',1,'R6CH','W07']]

In [37]:
# EAST
Weast = [[64,'W',1,'R1W1','W01'],
[65,'W',1,'R1W8','W08'],
[66,'W',1,'R1W5','W05'],
[67,'W',1,'R1W4','W04'],
[68,'W',1,'R1W6','W06'],
[69,'W',1,'R1W3','W03'],
[70,'W',1,'R1W7','W07'],
[71,'W',1,'R1W2','W02'],

[72,'W',1,'R2W1','W08'],
[73,'W',1,'R2W4','W04'],
[74,'W',1,'R2W3','W03'],
[75,'W',1,'R2W2','W07'],

[76,'W',1,'R3W1','W08'],
[77,'W',1,'R3W2','W07'],

[78,'W',1,'R4W1','W07']]

# WEST
Wwest = [[79,'W',1,'R1X1','X01'],
[80,'W',1,'R1X8','X08'],
[81,'W',1,'R1X5','X05'],
[82,'W',1,'R1X4','X04'],
[83,'W',1,'R1X6','X06'],
[84,'W',1,'R1X3','X03'],
[85,'W',1,'R1X7','X07'],
[86,'W',1,'R1X2','X02'],

[87,'W',1,'R2X1','X08'],
[88,'W',1,'R2X4','X05'],
[89,'W',1,'R2X3','X06'],
[90,'W',1,'R2X2','X02'],

[91,'W',1,'R3X1','X08'],
[92,'W',1,'R3X2','X06'],

[93,'W',1,'R4X1','X08']]

# SOUTH
Wsouth = [[94,'W',1,'R1Y1','Y01'],
[95,'W',1,'R1Y8','Y09'],
[96,'W',1,'R1Y5','Y05'],
[97,'W',1,'R1Y4','Y04'],
[98,'W',1,'R1Y6','Y06'],
[99,'W',1,'R1Y3','Y03'],
[100,'W',1,'R1Y7','Y07'],
[101,'W',1,'R1Y2','Y15'],

[102,'W',1,'R2Y1','Y09'],
[103,'W',1,'R2Y4','Y04'],
[104,'W',1,'R2Y3','Y03'],
[105,'W',1,'R2Y2','Y07'],

[106,'W',1,'R3Y1','Y04'],
[107,'W',1,'R3Y2','Y03'],

[108,'W',1,'R4Y1','Y03']]

# MIDWEST
Wmidwest = [[109,'W',1,'R1Z1','Z01'],
[110,'W',1,'R1Z8','Z09'],
[111,'W',1,'R1Z5','Z05'],
[112,'W',1,'R1Z4','Z04'],
[113,'W',1,'R1Z6','Z06'],
[114,'W',1,'R1Z3','Z03'],
[115,'W',1,'R1Z7','Z07'],
[116,'W',1,'R1Z2','Z15'],

[117,'W',1,'R2Z1','Z01'],
[118,'W',1,'R2Z4','Z05'],
[119,'W',1,'R2Z3','Z06'],
[120,'W',1,'R2Z2','Z15'],

[121,'W',1,'R3Z1','Z05'],
[122,'W',1,'R3Z2','Z06'],

[123,'W',1,'R4Z1','Z06']]

# FINAL FOUR
Wff = [[124,'W',1,'R5WX','W07'],
[125,'W',1,'R5YZ','Z06'],
[126,'W',1,'R6CH','W07']]

In [38]:
east = pd.DataFrame(east, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
west = pd.DataFrame(west, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
south = pd.DataFrame(south, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
midwest = pd.DataFrame(midwest, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
ff = pd.DataFrame(ff, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
Weast = pd.DataFrame(Weast, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
Wwest = pd.DataFrame(Wwest, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
Wsouth = pd.DataFrame(Wsouth, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
Wmidwest = pd.DataFrame(Wmidwest, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])
Wff = pd.DataFrame(Wff, columns = ['RowId', 'Tournament', 'Bracket', 'Slot', 'Team'])

In [39]:
submission = pd.concat([submission, east, west, south, midwest, ff, Weast, Wwest, Wsouth, Wmidwest, Wff])

In [40]:
submission.to_csv('submission.csv', index =False)

In [41]:
pd.read_csv('/kaggle/working/submission.csv')

,RowId,Tournament,Bracket,Slot,Team
0,1,M,1,R1W1,W01
1,2,M,1,R1W8,W08
2,3,M,1,R1W5,W05
3,4,M,1,R1W4,W04
4,5,M,1,R1W6,W06
...,...,...,...,...,...
121,122,W,1,R3Z2,Z06
122,123,W,1,R4Z1,Z06
123,124,W,1,R5WX,W07
124,125,W,1,R5YZ,Z06
